# Discovery

Required packages: vfb-connect and python-catmaid (pymaid & navis)

In [1]:
!pip install vfb-connect --upgrade
!pip install python-catmaid --upgrade

  Obtaining dependency information for vfb-connect from https://files.pythonhosted.org/packages/d8/d7/0c7d60d7af8a1f7afc81d46d5805e501721fec36c3b4f726ae958f8671c9/vfb_connect-1.2.12-py3-none-any.whl.metadata
  Attempting uninstall: vfb-connect
    Found existing installation: vfb-connect 1.2.12.dev34+0ddbd15.dirty
    Uninstalling vfb-connect-1.2.12.dev34+0ddbd15.dirty:
      Successfully uninstalled vfb-connect-1.2.12.dev34+0ddbd15.dirty
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.5/177.5 kB 2.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 5.3 MB/s eta 0:00:0000:0100:01
  Using cached ncollpyde-0.19.0-cp36-abi3-macosx_10_9_x86_64.macosx_11_0_arm64.macosx_10_9_universal2.whl (540 kB)
  Obtaining dependency information for igraph!=0.10.0,!=0.10.1 from https://files.pythonhosted.org/packages/40/d6/7ae77b847631bedac753db9040ac1d230bd9405ad3a9d0b2ded66e1a8c15/igraph-0.10.8-cp37-cp37m-macosx_10_9_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

### How to find neurons across datasets

VirtualFlyBrain integrates images and connectomics profiles of neurons from many sources.  It classifies and records their properties using a standard, queryable classification ([The Drosophila Anatomy Ontology](https://www.ebi.ac.uk/ols/ontologies/fbbt)). This standardises the names of neuron types across sources, so you don't need to worry about differences in nomenclature uses and supports queries for neurons by their classification. 

In [2]:
# Import libs and initialise API objects
from vfb_connect.cross_server_tools import VfbConnect
import pandas as pd
vc = VfbConnect()

import pymaid
import navis

navis.set_pbars(jupyter=False)
pymaid.set_pbars(jupyter=False)

# Connect to the VFB CATMAID server hosting the FAFB data
rm = pymaid.connect_catmaid(server="https://fafb.catmaid.virtualflybrain.org/", api_token=None, max_threads=10)

# Test call to see if connection works 
print(f'Server is running CATMAID version {rm.catmaid_version}')


INFO  : Global CATMAID instance set. Caching is ON. (pymaid)


Server is running CATMAID version 2020.02.15-102-gce1dee4


### Finds neurons by type (classification) across datasets

We can use the `vc.get_instances` method in combination with the name of a neuron type on VFB to find individual neurons from multiple sources.

Use the search tool on [VFB](http://virtualflybrain.org) to find neuron types by name or synonym:

<img src="https://user-images.githubusercontent.com/112839/109564687-535c9380-7ad9-11eb-80e9-5a5bc21cd915.png" width=30% height=30%>
<img src="https://user-images.githubusercontent.com/112839/109565128-eac1e680-7ad9-11eb-9649-1ec55a298a1b.png" width=30% height=30%>

Use either the full name or the Symbol to query for neurons:

In [3]:
DA3adPN = vc.get_instances("adult Drosulfakinin neuron", summary=True)
pd.DataFrame.from_records(DA3adPN)

,label,symbol,id,tags,parents_label,parents_id,data_source,accession,templates,dataset,license
0,DSKMP1A(PVM02)_L (FlyEM-HB:1260833150),1260833150,VFB_jrchjti3,Entity|Adult|Anatomy|Cell|Individual|Nervous_s...,adult Drosulfakinin neuron,FBbt_00048999,neuprint_JRC_Hemibrain_1point1,1260833150,JRC_FlyEM_Hemibrain|JRC2018Unisex,Xu2020NeuronsV1point1,https://creativecommons.org/licenses/by/4.0/le...
1,DSKMP3_R (FlyEM-HB:327937328),327937328,VFB_jrchjti6,Entity|Adult|Anatomy|Cell|Individual|Nervous_s...,adult Drosulfakinin neuron,FBbt_00048999,neuprint_JRC_Hemibrain_1point1,327937328,JRC_FlyEM_Hemibrain|JRC2018Unisex,Xu2020NeuronsV1point1,https://creativecommons.org/licenses/by/4.0/le...
2,DSKMP3_R (FlyEM-HB:328559607),328559607,VFB_jrchjti7,Entity|Adult|Anatomy|Cell|Individual|Nervous_s...,adult Drosulfakinin neuron,FBbt_00048999,neuprint_JRC_Hemibrain_1point1,328559607,JRC_FlyEM_Hemibrain|JRC2018Unisex,Xu2020NeuronsV1point1,https://creativecommons.org/licenses/by/4.0/le...
3,DSKMP1A_R (FlyEM-HB:1135837629),1135837629,VFB_jrchjti2,Entity|Adult|Anatomy|Cell|Individual|Nervous_s...,adult Drosulfakinin neuron,FBbt_00048999,neuprint_JRC_Hemibrain_1point1,1135837629,JRC2018Unisex|JRC_FlyEM_Hemibrain,Xu2020NeuronsV1point1,https://creativecommons.org/licenses/by/4.0/le...
4,DSKMP1B_R (FlyEM-HB:1352077058),1352077058,VFB_jrchjti4,Entity|Adult|Anatomy|Cell|Individual|Nervous_s...,adult Drosulfakinin neuron,FBbt_00048999,neuprint_JRC_Hemibrain_1point1,1352077058,JRC_FlyEM_Hemibrain|JRC2018Unisex,Xu2020NeuronsV1point1,https://creativecommons.org/licenses/by/4.0/le...
5,DSKMP1B(PVM02)_L (FlyEM-HB:1011184205),1011184205,VFB_jrchjti5,Entity|Adult|Anatomy|Cell|Individual|Nervous_s...,adult Drosulfakinin neuron,FBbt_00048999,neuprint_JRC_Hemibrain_1point1,1011184205,JRC_FlyEM_Hemibrain|JRC2018Unisex,Xu2020NeuronsV1point1,https://creativecommons.org/licenses/by/4.0/le...


### Find neurons by location

We can use the same method to search for neurons by location, using simple queries.

In [4]:
# Find neurons by location. The following query works across multiple data sources and both sides of the brain.  
# Results may be incomplete & may include minor overlap inferred from low synapse counts

neurons_in_DA3 = vc.get_instances("'neuron' that 'overlaps' some 'antennal lobe glomerulus DA3'", summary=True)
neurons_in_DA3_tab = pd.DataFrame.from_records(neurons_in_DA3)
neurons_in_DA3_tab[0:5]


Running query: FBbt:00005106 that RO:0002131 some FBbt:00003934
Query URL: http://owl.virtualflybrain.org/kbs/vfb/instances?object=FBbt%3A00005106+that+RO%3A0002131+some+FBbt%3A00003934&prefixes=%7B%22FBbt%22%3A+%22http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FFBbt_%22%2C+%22RO%22%3A+%22http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FRO_%22%2C+%22BFO%22%3A+%22http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FBFO_%22%7D&direct=False
Query results: 159


,label,symbol,id,tags,parents_label,parents_id,data_source,accession,templates,dataset,license
0,ORN_DA1_R (FlyEM-HB:1703682799),1703682799,VFB_jrchk15e,Entity|Adult|Anatomy|Cell|Chemosensory_system|...,adult olfactory receptor neuron Or67d,FBbt_00067031,neuprint_JRC_Hemibrain_1point1,1703682799,JRC_FlyEM_Hemibrain|JRC2018Unisex,Xu2020NeuronsV1point1,https://creativecommons.org/licenses/by/4.0/le...
1,ORN_DL4_R (FlyEM-HB:1671271059),1671271059,VFB_jrchk1km,Entity|Adult|Anatomy|Cell|Chemosensory_system|...,adult olfactory receptor neuron Or49a/85f,FBbt_00067055,neuprint_JRC_Hemibrain_1point1,1671271059,JRC_FlyEM_Hemibrain|JRC2018Unisex,Xu2020NeuronsV1point1,https://creativecommons.org/licenses/by/4.0/le...
2,ORN_DA1_R (FlyEM-HB:1735054607),1735054607,VFB_jrchk13v,Entity|Adult|Anatomy|Cell|Chemosensory_system|...,adult olfactory receptor neuron Or67d,FBbt_00067031,neuprint_JRC_Hemibrain_1point1,1735054607,JRC2018Unisex|JRC_FlyEM_Hemibrain,Xu2020NeuronsV1point1,https://creativecommons.org/licenses/by/4.0/le...
3,v2LN35_R (FlyEM-HB:1733056086),1733056086,VFB_jrchk8ew,Entity|Adult|Anatomy|Cell|Individual|Nervous_s...,adult antennal lobe local neuron v2LN35,FBbt_20003766,neuprint_JRC_Hemibrain_1point1,1733056086,JRC2018Unisex|JRC_FlyEM_Hemibrain,Xu2020NeuronsV1point1,https://creativecommons.org/licenses/by/4.0/le...
4,lLN2T_a(Tortuous)_R (FlyEM-HB:5813032595),5813032595,VFB_jrchk8bd,Entity|Adult|Anatomy|Cell|Individual|Nervous_s...,adult antennal lobe local neuron lLN2T,FBbt_00049816,neuprint_JRC_Hemibrain_1point1,5813032595,JRC2018Unisex|JRC_FlyEM_Hemibrain,Xu2020NeuronsV1point1,https://creativecommons.org/licenses/by/4.0/le...


In [5]:
# Find local interneurons (intrinsic neurons) of the AL, overlapping DA3:

local_in_DA3 = vc.get_instances("'local interneuron of adult antennal lobe' that 'overlaps' some 'antennal lobe glomerulus DA3'",
                                summary=True)
local_in_DA3_tab = pd.DataFrame.from_records(local_in_DA3)
local_in_DA3_tab

Running query: FBbt:00007390 that RO:0002131 some FBbt:00003934
Query URL: http://owl.virtualflybrain.org/kbs/vfb/instances?object=FBbt%3A00007390+that+RO%3A0002131+some+FBbt%3A00003934&prefixes=%7B%22FBbt%22%3A+%22http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FFBbt_%22%2C+%22RO%22%3A+%22http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FRO_%22%2C+%22BFO%22%3A+%22http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FBFO_%22%7D&direct=False
Query results: 53


,label,symbol,id,tags,parents_label,parents_id,data_source,accession,templates,dataset,license
0,v2LN35_R (FlyEM-HB:1733056086),1733056086,VFB_jrchk8ew,Entity|Adult|Anatomy|Cell|Individual|Nervous_s...,adult antennal lobe local neuron v2LN35,FBbt_20003766,neuprint_JRC_Hemibrain_1point1,1733056086,JRC2018Unisex|JRC_FlyEM_Hemibrain,Xu2020NeuronsV1point1,https://creativecommons.org/licenses/by/4.0/le...
1,lLN2T_a(Tortuous)_R (FlyEM-HB:5813032595),5813032595,VFB_jrchk8bd,Entity|Adult|Anatomy|Cell|Individual|Nervous_s...,adult antennal lobe local neuron lLN2T,FBbt_00049816,neuprint_JRC_Hemibrain_1point1,5813032595,JRC2018Unisex|JRC_FlyEM_Hemibrain,Xu2020NeuronsV1point1,https://creativecommons.org/licenses/by/4.0/le...
2,lLN2T_d(Tortuous)_R (FlyEM-HB:1667251683),1667251683,VFB_jrchk8bp,Entity|Adult|Anatomy|Cell|Individual|Nervous_s...,adult antennal lobe local neuron lLN2T,FBbt_00049816,neuprint_JRC_Hemibrain_1point1,1667251683,JRC_FlyEM_Hemibrain|JRC2018Unisex,Xu2020NeuronsV1point1,https://creativecommons.org/licenses/by/4.0/le...
3,lLN2P_b(Patchy)_R (FlyEM-HB:1946178096),1946178096,VFB_jrchk8au,Entity|Adult|Anatomy|Cell|Individual|Nervous_s...,adult antennal lobe local neuron lLN2P,FBbt_00049813,neuprint_JRC_Hemibrain_1point1,1946178096,JRC2018Unisex|JRC_FlyEM_Hemibrain,Xu2020NeuronsV1point1,https://creativecommons.org/licenses/by/4.0/le...
4,lLN2T_a(Tortuous)_R (FlyEM-HB:1762354941),1762354941,VFB_jrchk8bg,Entity|Adult|Anatomy|Cell|Individual|Nervous_s...,adult antennal lobe local neuron lLN2T,FBbt_00049816,neuprint_JRC_Hemibrain_1point1,1762354941,JRC_FlyEM_Hemibrain|JRC2018Unisex,Xu2020NeuronsV1point1,https://creativecommons.org/licenses/by/4.0/le...
5,lLN1_c_R (FlyEM-HB:1547454812),1547454812,VFB_jrchk8ai,Entity|Adult|Anatomy|Cell|Individual|Nervous_s...,adult antennal lobe local neuron lLN1,FBbt_00007395,neuprint_JRC_Hemibrain_1point1,1547454812,JRC_FlyEM_Hemibrain|JRC2018Unisex,Xu2020NeuronsV1point1,https://creativecommons.org/licenses/by/4.0/le...
6,lLN1_b_R (FlyEM-HB:5813054622),5813054622,VFB_jrchk8a8,Entity|Adult|Anatomy|Cell|Individual|Nervous_s...,adult antennal lobe local neuron lLN1,FBbt_00007395,neuprint_JRC_Hemibrain_1point1,5813054622,JRC_FlyEM_Hemibrain|JRC2018Unisex,Xu2020NeuronsV1point1,https://creativecommons.org/licenses/by/4.0/le...
7,lLN2T_a(Tortuous)_R (FlyEM-HB:5813056598),5813056598,VFB_jrchk8bi,Entity|Adult|Anatomy|Cell|Individual|Nervous_s...,adult antennal lobe local neuron lLN2T,FBbt_00049816,neuprint_JRC_Hemibrain_1point1,5813056598,JRC2018Unisex|JRC_FlyEM_Hemibrain,Xu2020NeuronsV1point1,https://creativecommons.org/licenses/by/4.0/le...
8,lLN16b_R (FlyEM-HB:1887168462),1887168462,VFB_jrchk89w,Entity|Adult|Anatomy|Cell|Individual|Nervous_s...,adult antennal lobe local neuron lLN16B,FBbt_20003752,neuprint_JRC_Hemibrain_1point1,1887168462,JRC2018Unisex|JRC_FlyEM_Hemibrain,Xu2020NeuronsV1point1,https://creativecommons.org/licenses/by/4.0/le...
9,lLN2T_e(Tortuous)_R (FlyEM-HB:1640922516),1640922516,VFB_jrchk8bs,Entity|Adult|Anatomy|Cell|Individual|Nervous_s...,adult antennal lobe local neuron lLN2T,FBbt_00049816,neuprint_JRC_Hemibrain_1point1,1640922516,JRC2018Unisex|JRC_FlyEM_Hemibrain,Xu2020NeuronsV1point1,https://creativecommons.org/licenses/by/4.0/le...


In [6]:
# Find neurons by dataset/paper - on CATMAID

bates = pymaid.find_neurons(annotations='Paper: Bates and Schlegel et al 2020')
bates

INFO  : Found 584 neurons matching the search parameters (pymaid)


,type,name,skeleton_id,n_nodes,n_connectors,n_branches,n_leafs,cable_length,soma,units
0,CatmaidNeuron,Uniglomerular mALT DA1 lPN 57316 2863105 ML,2863104,NA,NA,NA,NA,NA,NA,1 nanometer
1,CatmaidNeuron,Uniglomerular mALT DA3 adPN 57350 HG,57349,NA,NA,NA,NA,NA,NA,1 nanometer
...,...,...,...,...,...,...,...,...,...,...
582,CatmaidNeuron,Multiglomerular mlALT vPN VM7d+VM5d+DC4+6 LTS ...,4624378,NA,NA,NA,NA,NA,NA,1 nanometer
583,CatmaidNeuron,Uniglomerular mALT DL2d adPN 57342 ML,57341,NA,NA,NA,NA,NA,NA,1 nanometer


In [7]:
# Inspect what datasets are available on VFB

ds = vc.neo_query_wrapper.get_datasets(summary=True)
ds_tab = pd.DataFrame.from_records(ds)
ds_tab.sort_values(by=['id'])

,label,symbol,id,tags,description,miniref,FlyBase,PMID,DOI
67,EM L1 Andrade et al. 2019,,Andrade2019,Entity|DataSet|Individual|has_image,[L1 EM reconstructed neurons from Andrade et a...,"Andrade et al., 2019, Curr. Biol. 29(3): 412--...",FBrf0241389,30661802,10.1016/j.cub.2018.12.012
29,MBONs and split-GAL4 lines that target them (A...,,Aso2014,Entity|DataSet|Individual|has_image,[],"Aso et al., 2014, eLife 3: e04577",FBrf0227179,25535793,10.7554/eLife.04577
64,split-GAL4 lines for dopaminergic neurons (Aso...,,AsoRubin2016,Entity|DataSet|Individual|has_image,[For comparison of the properties of memories ...,"Aso and Rubin, 2016, eLife 5: e16135",FBrf0233230,27441388,10.7554/eLife.16135
144,EM FAFB Baltruschat et al 2021,,Baltruschat2021,Entity|DataSet|Individual|has_image,[FAFB EM reconstructed neurons from Baltruscha...,"Baltruschat et al., 2021, Cell Rep. 34(11): 10...",FBrf0248487,33730583,10.1016/j.celrep.2021.108871
28,"EM L1 Barnes et al., 2022",,Barnes2022,Entity|DataSet|Individual|L1EM,[L1 EM reconstructed neurons from Barnes et al...,"Barnes et al., 2022, PLoS ONE 17(4): e0266064",FBrf0253082,35377898,10.1371/journal.pone.0266064
...,...,...,...,...,...,...,...,...,...
88,"EM L1 Zarin, Mark et al. 2019",,Zarin2019,Entity|DataSet|Individual|has_image,"[L1 EM reconstructed neurons from Zarin, Mark ...","Zarin, Mark et al., 2019",,,10.1101/617977
86,"EM FAFB Zhao et al., 2023",,Zhao2023,Entity|DataSet|Individual|FAFB,[FAFB EM reconstructed neurons from Zhao et al...,,,,
43,EM FAFB Zheng et al 2018,,Zheng2018,Entity|DataSet|Individual|has_image,[FAFB EM reconstructed neurons from Zheng et a...,"Zheng et al., 2018, Cell 174(3): 730--743.e22",FBrf0239557,30033368,10.1016/j.cell.2018.06.019
44,EM FAFB Zheng et al 2020 (published 2022),,Zheng2020,Entity|DataSet|Individual|has_image,[FAFB EM reconstructed neurons from Zheng et a...,"Zheng et al., 2022, Curr. Biol. 32(15): 3334--...",FBrf0254218,35797998,10.1016/j.cub.2022.06.031


In [8]:
sayin_tab = pd.DataFrame.from_records(vc.get_instances_by_dataset('Sayin2019', summary=True))
sayin_tab

,label,symbol,id,tags,parents_label,parents_id,data_source,accession,templates,dataset,license
0,OA-VPM3 (FAFB:1329078),,VFB_001001dr,Entity|Adult|Anatomy|Cell|Individual|Nervous_s...,octopaminergic VPM3 neuron,FBbt_00110151,catmaid_fafb,1329078,JRC2018Unisex,Sayin2019,https://creativecommons.org/licenses/by-sa/4.0...
1,OA-VPM4 (FAFB:1191261),,VFB_001001dq,Entity|Adult|Anatomy|Cell|Individual|Nervous_s...,octopaminergic VPM4 neuron,FBbt_00110152,catmaid_fafb,1191261,JRC2018Unisex,Sayin2019,https://creativecommons.org/licenses/by-sa/4.0...


In [9]:
vc.get_connected_neurons_by_type(upstream_type='LNd', downstream_type='adult descending neuron', weight=20)

,upstream_neuron_id,upstream_neuron_name,weight,downstream_neuron_id,downstream_neuron_name,upstream_class,downstream_class,up_data_source,up_accession,down_source,down_accession
0,VFB_jrchjzxx,LNd_R (FlyEM-HB:5813021192),25,VFB_jrchjthm,DNp27_R (FlyEM-HB:1072063538),LNd neuron,descending neuron of the posterior brain DNp27,None,None,None,None


In [10]:
# Intra pacemaker neuron neuron connections

vc.get_connected_neurons_by_type(upstream_type='pacemaker neuron', downstream_type='pacemaker neuron', weight=20)

,upstream_neuron_id,upstream_neuron_name,weight,downstream_neuron_id,downstream_neuron_name,upstream_class,downstream_class,up_data_source,up_accession,down_source,down_accession
0,VFB_jrchk089,LPN_R (FlyEM-HB:480029788),43,VFB_jrchk08a,LPN_R (FlyEM-HB:450034902),LP neuron,LP neuron,None,None,None,None
1,VFB_jrchk08a,LPN_R (FlyEM-HB:450034902),30,VFB_jrchk089,LPN_R (FlyEM-HB:480029788),LP neuron,LP neuron,None,None,None,None
2,VFB_jrchjtf3,DN1pA_R (FlyEM-HB:387166379),30,VFB_jrchjzxy,LNd_R (FlyEM-HB:5813069648),DN1p neuron,LNd neuron,None,None,None,None
3,VFB_jrchjtf1,DN1pA_R (FlyEM-HB:325529237),30,VFB_jrchjzxy,LNd_R (FlyEM-HB:5813069648),DN1p neuron,LNd neuron,None,None,None,None
4,VFB_jrchjtey,DN1a_R (FlyEM-HB:5813022274),75,VFB_jrchjzxy,LNd_R (FlyEM-HB:5813069648),DN1a neuron,LNd neuron,None,None,None,None
5,VFB_jrchjtex,DN1a_R (FlyEM-HB:264083994),79,VFB_jrchjzxy,LNd_R (FlyEM-HB:5813069648),DN1a neuron,LNd neuron,None,None,None,None
6,VFB_jrchjtf2,DN1pA_R (FlyEM-HB:387944118),34,VFB_jrchjzxy,LNd_R (FlyEM-HB:5813069648),DN1p neuron,LNd neuron,None,None,None,None
7,VFB_jrchk8e0,5th s-LNv (FlyEM-HB:511051477),40,VFB_jrchjzxy,LNd_R (FlyEM-HB:5813069648),s-LNv neuron,LNd neuron,None,None,None,None
8,VFB_jrchjtf0,DN1pA_R (FlyEM-HB:324846570),37,VFB_jrchjzxy,LNd_R (FlyEM-HB:5813069648),DN1p neuron,LNd neuron,None,None,None,None
9,VFB_jrchjzxy,LNd_R (FlyEM-HB:5813069648),29,VFB_jrchjzxw,LNd_R (FlyEM-HB:5813056917),LNd neuron,LNd neuron,None,None,None,None


In [11]:

vc.get_connected_neurons_by_type(upstream_type='adult neuron', downstream_type='adult Drosulfakinin neuron', weight=20)

,upstream_neuron_id,upstream_neuron_name,weight,downstream_neuron_id,downstream_neuron_name,upstream_class,downstream_class,up_data_source,up_accession,down_source,down_accession
0,VFB_jrchk5qg,SLP244_R (FlyEM-HB:297533212),33,VFB_jrchjti6,DSKMP3_R (FlyEM-HB:327937328),adult superior lateral protocerebrum neuron 244,adult Drosulfakinin neuron,None,None,None,None
1,VFB_jrchk5qf,SLP244_R (FlyEM-HB:298214577),27,VFB_jrchjti6,DSKMP3_R (FlyEM-HB:327937328),adult superior lateral protocerebrum neuron 244,adult Drosulfakinin neuron,None,None,None,None
2,VFB_jrchjyu7,LHAV3j1_R (FlyEM-HB:297851977),24,VFB_jrchjti6,DSKMP3_R (FlyEM-HB:327937328),adult lateral horn AV3j1 neuron,adult Drosulfakinin neuron,None,None,None,None
3,VFB_jrchk64p,SMP001_R (FlyEM-HB:5813056887),22,VFB_jrchjti6,DSKMP3_R (FlyEM-HB:327937328),adult superior medial protocerebrum neuron 001,adult Drosulfakinin neuron,None,None,None,None
4,VFB_jrchk0fd,MBON23(a2sp)_R (FlyEM-HB:423382015),21,VFB_jrchjti6,DSKMP3_R (FlyEM-HB:327937328),mushroom body output neuron 23,adult Drosulfakinin neuron,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
56,VFB_jrchjti6,DSKMP3_R (FlyEM-HB:327937328),39,VFB_jrchjti4,DSKMP1B_R (FlyEM-HB:1352077058),adult Drosulfakinin neuron,adult Drosulfakinin neuron,None,None,None,None
57,VFB_jrchk8dv,pC1c_R (FlyEM-HB:267551639),84,VFB_jrchjti4,DSKMP1B_R (FlyEM-HB:1352077058),adult doublesex pC1 (female) neuron,adult Drosulfakinin neuron,None,None,None,None
58,VFB_jrchk5jv,SLP152_R (FlyEM-HB:487812620),28,VFB_jrchjti4,DSKMP1B_R (FlyEM-HB:1352077058),adult superior lateral protocerebrum neuron 152,adult Drosulfakinin neuron,None,None,None,None
59,VFB_jrchk5eo,SLP066_R (FlyEM-HB:327843160),42,VFB_jrchjti4,DSKMP1B_R (FlyEM-HB:1352077058),adult superior lateral protocerebrum neuron 066,adult Drosulfakinin neuron,None,None,None,None


### Transcriptomics Queries

# Finding single cell transcriptomics data for a given cell type

In [12]:

profile = vc.get_transcriptomic_profile(cell_type = 'adult olfactory receptor neuron')
print(profile.head())

                                  cell_type   cell_type_id       sample_sex  \
0  adult antennal olfactory receptor neuron  FBbt_00007352  female organism   
1  adult antennal olfactory receptor neuron  FBbt_00007352    male organism   
2  adult antennal olfactory receptor neuron  FBbt_00007352             None   
3  adult antennal olfactory receptor neuron  FBbt_00007352             None   
4  adult antennal olfactory receptor neuron  FBbt_00007352    male organism   

                                       sample_tissue  \
0  [antenna, adult tracheal system, wing, haltere...   
1  [adult head, adult, wing, proboscis, testis, m...   
2  [leg, adult, adult fat body, adult abdomen, ad...   
3  [leg, adult, adult fat body, adult abdomen, ad...   
4  [adult head, adult, wing, proboscis, testis, m...   

                                            ref     gene      gene_id  \
0  Li et al., 2022, Science 375(6584): eabk2432  14-3-3ε  FBgn0020238   
1  Li et al., 2022, Science 375(6584): eab

These results can be filtered by gene function - first find available gene functions

In [13]:
functions = vc.get_gene_function_filters()
print(functions)

['Acetylcholine_receptor', 'Calcium_binding', 'Dopamine_receptor', 'Enzyme', 'GABA_receptor', 'GPCR', 'Glutamate_receptor', 'Gustatory_receptor', 'Histamine_receptor', 'Hormone', 'Ion_channel', 'Mechanosensory_ion_channel', 'Neuropeptide', 'Octopamine_receptor', 'Odorant_binding', 'Olfactory_receptor', 'Peptide_or_protein_hormone_receptor', 'Photoreceptor', 'Serotonin_receptor', 'Thermosensory_ion_channel', 'Transcription_factor', 'Tyramine_receptor']


Then run transcriptomics query speicfying a gene function to filter on

In [14]:
filtered_profile = vc.get_transcriptomic_profile(cell_type = 'adult olfactory receptor neuron', gene_type='Olfactory_receptor')
print(filtered_profile.head())

                         cell_type   cell_type_id       sample_sex  \
0  adult olfactory receptor neuron  FBbt_00007386  female organism   
1  adult olfactory receptor neuron  FBbt_00007386    male organism   
2  adult olfactory receptor neuron  FBbt_00007386             None   
3  adult olfactory receptor neuron  FBbt_00007386  female organism   
4  adult olfactory receptor neuron  FBbt_00007386             None   

                                       sample_tissue  \
0                                          [antenna]   
1  [adult head, adult, wing, proboscis, testis, m...   
2                                          [antenna]   
3                                       [adult head]   
4                                       [adult head]   

                                            ref  gene      gene_id  \
0  Li et al., 2022, Science 375(6584): eabk2432  Orco  FBgn0037324   
1  Li et al., 2022, Science 375(6584): eabk2432  Orco  FBgn0037324   
2  Li et al., 2022, Science 375(